In [ ]:
!pip install -U openai-whisper

In [ ]:
!pip install moviepy

In [ ]:
import whisper

whisper_model = whisper.load_model("large-v3")

In [ ]:
import os
from moviepy.editor import VideoFileClip

In [ ]:
from openai import OpenAI

## video から抽出する場合

In [ ]:
# Set the paths
video_path = "/content/test_video.mp4"  # Replace with your actual video file name
output_audio_path = "/content/test_audio.wav"

# Extract audio from the video
video = VideoFileClip(video_path)
video.audio.write_audiofile(output_audio_path)

### gemini api

In [ ]:
from google import genai
from google.genai import types

In [ ]:
from google.colab import userdata
GEMINI_KEY = userdata.get("GEMINI_API_KEY")
# 利用するモデルを選択

In [ ]:
client = genai.Client(api_key=GEMINI_KEY, vertexai=False)
myfile = client.files.upload(file='/content/small_split_data.wav')
prompt = 'Generate a transcript of the speech.'

In [ ]:
myfile.uri

In [ ]:
prompt = """
保育士教育として、不適切な保育およびよい保育の検知をおこないます。
日本語のオーディオファイルを解析し、子ども家庭庁の「保育所等における虐待等の防止及び 発生時の対応等に関するガイドライン」を参考にして、不適切な保育の検知をしてください。

### System Instructions:
1. 話者分離: 先生と子供の発言箇所を分離します。
2. 会話内容の検出: 先生の発言箇所のみを解析対象とし、発言内容を解析します。
3. フィルタリング: 会話内容および声のトーンから解析できる感情を利用し、不適切、または権利の侵害をしている箇所を抽出します。抽出箇所が存在しない場合は、空のリストを返却します。

### Detection Criteria:
- 権威の濫用（威圧的態度）: 相手を怖がらせたり、無理やり従わせようとしたりする強圧的な口調。
- 尊厳の侵害: 皮肉を言ったり、相手を軽んじたり、あざ笑うような言葉遣い。
- 恐怖によるコントロール: 子供をコントロールするために、大声（怒鳴り声）を出したり、脅し文句を使ったりすること。

### Audio Context:
- 言語: 日本語
- 環境: 保育園/幼稚園
  """

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=[
      types.Content(
        parts=[
          types.Part(
            file_data=types.FileData(
              file_uri=myfile.uri
            )
          ),
          types.Part(
            text=prompt
          )
        ]
      )
    ],
    config=types.GenerateContentConfig(
      response_mime_type="application/json",
      response_schema = types.Schema(
      type=types.Type.OBJECT,
      properties={
          "segments": types.Schema(
              type=types.Type.ARRAY,
              items=types.Schema(
                  type=types.Type.OBJECT,
                  properties={
                      # 1. 基本データ (Diarization)
                      "start_timestamp": types.Schema(type=types.Type.INTEGER, description="開始時間 (ms)"),
                      "end_timestamp": types.Schema(type=types.Type.INTEGER, description="終了時間 (ms)"),
                      "speaker": types.Schema(
                          type=types.Type.STRING,
                          enum=["teacher", "child"],
                          description="話者の種別"
                      ),
                      "transcript": types.Schema(type=types.Type.STRING, description="発話内容"),

                      # 2. 先生の場合のみ詳細に埋めるデータ (Analysis)
                      "confidence_level": types.Schema(type=types.Type.NUMBER, description="文字起こしの確信度 (0.0-1.0)"),
                      "tone": types.Schema(
                          type=types.Type.STRING,
                          enum=["gentle", "strict", "praising", "instructional", "neutral"], # 業務に合わせて定義
                          description="先生の口調"
                      ),
                      "category": types.Schema(
                        type=types.Type.STRING,
                        enum=["authority_abuse", "dignity_violation", "control_by_fear", "none"],
                        description="該当する違反カテゴリー"
                      ),
                      "reason": types.Schema(
                          type=types.Type.STRING,
                          description="その口調や発言に至った背景や理由の分析"
                      ),
                  },
                  required=["start_timestamp", "end_timestamp", "speaker", "transcript", "category"]
              )
          )
      },
      required=["segments"]
      )
    )
)


print(response.text)

In [ ]:
response

In [ ]:
prompt = """
日本語のオーディオファイルを解析し、子ども家庭庁の「保育所等における虐待等の防止及び 発生時の対応等に関するガイドライン」を参考にして、不適切な保育の検知をしてください。

### System Instructions:
1. 話者分離: 先生と子供の発言箇所を分離します。
2. 会話内容の検出: 先生の発言箇所のみを解析対象とし、発言内容を解析します。
3. フィルタリング: 会話内容および声のトーンから解析できる感情を利用し、不適切、または権利の侵害をしている箇所を抽出します。抽出箇所が存在しない場合は、空のリストを返却します。

### Detection Criteria:
- 権威の濫用（威圧的態度）: 相手を怖がらせたり、無理やり従わせようとしたりする強圧的な口調。
- 尊厳の侵害: 皮肉を言ったり、相手を軽んじたり、あざ笑うような言葉遣い。
- 恐怖によるコントロール: 子供をコントロールするために、大声（怒鳴り声）を出したり、脅し文句を使ったりすること。

### Audio Context:
- 言語: 日本語
- 環境: 保育園/幼稚園
  """

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=[
      types.Content(
        parts=[
          types.Part(
            file_data=types.FileData(
              file_uri=myfile.uri
            )
          ),
          types.Part(
            text=prompt
          )
        ]
      )
    ],
    config=types.GenerateContentConfig(
      response_mime_type="application/json",
      response_schema = types.Schema(
      type=types.Type.OBJECT,
      properties={
          "segments": types.Schema(
              type=types.Type.ARRAY,
              items=types.Schema(
                  type=types.Type.OBJECT,
                  properties={
                      # 1. 基本データ (Diarization)
                      "start_timestamp": types.Schema(type=types.Type.INTEGER, description="開始時間 (ms)"),
                      "end_timestamp": types.Schema(type=types.Type.INTEGER, description="終了時間 (ms)"),
                      "speaker": types.Schema(
                          type=types.Type.STRING,
                          enum=["teacher", "child"],
                          description="話者の種別"
                      ),
                      "transcript": types.Schema(type=types.Type.STRING, description="発話内容"),

                      # 2. 先生の場合のみ詳細に埋めるデータ (Analysis)
                      "confidence_level": types.Schema(type=types.Type.NUMBER, description="文字起こしの確信度 (0.0-1.0)"),
                      "tone": types.Schema(
                          type=types.Type.STRING,
                          enum=["gentle", "strict", "praising", "instructional", "neutral"], # 業務に合わせて定義
                          description="先生の口調"
                      ),
                      "category": types.Schema(
                        type=types.Type.STRING,
                        enum=["authority_abuse", "dignity_violation", "control_by_fear", "none"],
                        description="該当する違反カテゴリー"
                      ),
                      "reason": types.Schema(
                          type=types.Type.STRING,
                          description="その口調や発言に至った背景や理由の分析"
                      ),
                  },
                  required=["start_timestamp", "end_timestamp", "speaker", "transcript", "category"]
              )
          )
      },
      required=["segments"]
      )
    )
)


print(response.text)

In [ ]:
import json
json.loads(response.text)

In [ ]:
prompt = """
Analyze the attached Japanese audio file and detect "Inappropriate Childcare" (不適切な保育) based on the Japanese Children and Families Agency guidelines.

### System Instructions:
1. **Multimodal Analysis**: Evaluate not only the words but also the emotional tone (e.g., aggression, yelling, sarcasm, pitch, and volume).
2. **Filtering**: Extract ONLY the segments that are potentially inappropriate or rights-violating. If no such interactions are found, return an empty list `[]`.
3. **JSON Structure**: Output the results strictly in the JSON format specified below.

### Detection Criteria:
- **Authority Abuse**: Intimidating or coercive tone.
- **Dignity Violation**: Sarcastic, belittling, or mocking language.
- **Control by Fear**: Using loud voices or threats to control children.

### Output Format (Strict JSON):
Return a JSON object with a key "incidents" containing an array of objects:
{
  "incidents": [
    {
      "timestamp": "MM:SS",
      "transcript": "該当する発言の日本語テキスト",
      "severity": "低 / 中 / 高",
      "emotional_tone": "Angry / Sarcastic / Threatening / Cold etc.",
      "reason": "ガイドライン抵触項目と、声のトーンから読み取れる状況の具体的説明"
    }
  ]
}

### Audio Context:
- Language: Japanese
- Environment: Kindergarten / Nursery school
  """

response = client.models.generate_content(
  model='gemini-2.5-flash', # 3系でよくなる
  contents=[prompt, myfile],
    config={
        "response_mime_type": "application/json",
    },
)

print(response.text)

In [ ]:
  response = client.models.generate_content(
    model="gemini-2.5-flash",


## 音声文字起こし

### whisper

In [ ]:
result = whisper_model.transcribe("/content/test_audio.wav",
                       initial_prompt="保育園、ゲーム、チーム",
                        language="ja")
print(result["text"])

In [ ]:
result = whisper_model.transcribe("/content/test_audio.wav",
                       #initial_prompt="保育園、ゲーム、チーム",
                        language="ja")
print(result["text"])

## 感情分析 - 没
- あるファイルについて感情分類するイメージ
- ちゃんと論文読まないときついかも
- speechbrain: Apache License 2.0 (商用OK）
- @See: https://huggingface.co/speechbrain/emotion-recognition-wav2vec2-IEMOCAP

In [ ]:
!pip install git+https://github.com/speechbrain/speechbrain.git@develop > /dev/null

In [ ]:
from speechbrain.pretrained.interfaces import foreign_class
classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")


In [ ]:
out_prob, score, index, text_lab = classifier.classify_file("speechbrain/emotion-recognition-wav2vec2-IEMOCAP/anger.wav")
print(text_lab)

In [ ]:
from speechbrain.inference.diarization import Speech_Emotion_Diarization
classifier = Speech_Emotion_Diarization.from_hparams(
    source="speechbrain/emotion-diarization-wavlm-large"
)


## 適当に録音 - 没

In [ ]:
# @title
!pip install sounddevice scipy > /dev/null

In [ ]:
# @title
from IPython.display import display, Javascript
from google.colab import output
import base64

def record_audio(filename='my_test_voice.wav'):
  js = Javascript('''
    async function recordAudio() {
      const div = document.createElement('div');
      const button = document.createElement('button');
      button.textContent = '録音開始';
      button.style.padding = '10px';
      document.body.appendChild(div);
      div.appendChild(button);

      const stream = await navigator.mediaDevices.getUserMedia({audio: true});
      const recorder = new MediaRecorder(stream);
      let chunks = [];

      button.onclick = () => {
        if (recorder.state === 'inactive') {
          recorder.start();
          button.textContent = '録音停止';
          button.style.background = 'red';
        } else {
          recorder.stop();
          button.textContent = '保存中...';
        }
      };

      const finished = new Promise(resolve => {
        recorder.onstop = () => {
          const blob = new Blob(chunks, {type: 'audio/wav'});
          const reader = new FileReader();
          reader.readAsDataURL(blob);
          reader.onloadend = () => resolve(reader.result);
        };
      });

      recorder.ondataavailable = e => chunks.push(e.data);
      return finished;
    }
  ''')
  display(js)
  data = output.eval_js('recordAudio()')
  binary = base64.b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  print(f"保存完了: {filename}")

# 実行
record_audio()

In [ ]:
# 精度が非常に怪しいため却下
# 2. 録音したファイルを正しいWAV（16kHz, モノラル）に変換
import os
os.system("ffmpeg -i my_test_voice.wav -vn -acodec pcm_s16le -ar 16000 -ac 1 fixed_voice.wav -y")

# 3. 変換したファイルで解析を実行
diary = classifier.diarize_file("fixed_voice.wav")
print(diary)

In [ ]:
result = whisper_model.transcribe("/content/fixed_voice.wav",
                        language="ja")
print(result["text"])

In [ ]:
from openai import OpenAI
from google.colab import userdata

In [ ]:
openai_client = OpenAI(api_key=userdata.get('OPENAI_TOKEN'))

In [ ]:
result_text = result["text"]

In [ ]:
response = openai_client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "system", "content": "あなたは保育の質の向上を支援する専門家です"},
        {"role": "user", "content": content}
    ],
    reasoning_effort="low",
    max_completion_tokens=1000
)

# 結果の出力
print(response.choices[0].message.content)

In [ ]:
response

In [ ]:
def create_prompt(exctract_text):
  return f"""
  以下の保育現場の会話ログを読み、
  こども家庭庁の『不適切な保育防止ガイドライン』に照らし合わせて、
  人権侵害や不適切な関わりの疑いがある箇所を抽出してください。

  会話ログ:
  「{exctract_text}」

  抽出条件:
  - 言葉の内容が威圧的、または子どもを蔑んでいる。
  - 恐怖や罰を用いて子どもをコントロールしようとしている。
  - 身体的、精神的な攻撃を示唆している。

  出力形式:
  [発言箇所]：
  [判定結果]：低/中/高
  [理由]：ガイドラインのどの項目に抵触するか説明。
  """


In [ ]:
def check_response(prompt, model="o3-mini", reasoning_effort="low"):
  response = openai_client.chat.completions.create(
      model=model,
      messages=[
          {"role": "system", "content": "あなたは保育の質の向上を支援する専門家です"},
          {"role": "user", "content": prompt}
      ],
      reasoning_effort=reasoning_effort,
      max_completion_tokens=1000
  )
  return response

In [ ]:
check_response(create_prompt("そんなことする子は先生もう嫌い。ママにもきっと嫌われちゃうよ"))